In [2]:
import pandas as pd
import requests
import datetime as dt
import json
import sys
import os

In [3]:
##get latest trailhead hotspot dataset

trailheadHotspots = pd.DataFrame()
trailheadHotspots = pd.read_csv('https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadHotspots/trailheadHotspots_Y2022_M1_D11.csv')
trailheadHotspots.head(5)

,locId,locName,countryCode,subnational1Code,subnational2Code,lat,lng,latestObsDt,numSpeciesAllTime,StopName
0,L10128988,Berntsen Park,US,US-WA,US-WA-033,47.538439,-122.043281,2021-12-15 11:46,15.0,EastSunsetWay
1,L10129014,"Confluence Park, Issaquah",US,US-WA,US-WA-033,47.536469,-122.039342,2022-01-08 12:54,44.0,EastSunsetWay
2,L8365620,Issaquah High School,US,US-WA,US-WA-033,47.522490,-122.028687,2021-12-17 08:00,58.0,EastSunsetWay
3,L8102503,Issaquah Salmon Hatchery,US,US-WA,US-WA-033,47.529503,-122.039512,2021-12-03 08:22,33.0,EastSunsetWay
4,L3352189,"Maple Street ponds, Issaquah",US,US-WA,US-WA-033,47.543027,-122.053715,2021-11-30 10:12,46.0,EastSunsetWay


In [4]:
##pull in trailhead reference as base for duplicate hotspot removal
trailheadRef = pd.DataFrame()
trailheadRef_json = 'https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadsRef.json'
response = requests.get(trailheadRef_json)
trailheadRef = pd.DataFrame(response.json())

trailheadRef.head()

,Route,StopName,Address,Latitude,Longitude
0,IssaquahAlps,EastSunsetWay,"661-831 E Sunset Way, Issaquah, WA 98027",47.529635,-122.025119
1,IssaquahAlps,HighSchool,"Parking lot, The Rainier Trail, Issaquah, WA 9...",47.519345,-122.029801
2,IssaquahAlps,ChiricoTrail_PooPooPoint,"11400 Issaquah-Hobart Road Southeast, Issaquah...",47.499949,-122.021730
3,IssaquahAlps,SquakMountain,"13201 Squak Mountain Rd SE, Issaquah, WA 98027",47.481465,-122.053997
4,IssaquahAlps,MargaretsWay,"190th Ave SE, Issaquah, WA 98027",47.506620,-122.086660


In [5]:
trailheadHotspots['StopName'].value_counts()

DiscoveryParkSouth          26
DiscoveryParkNorth          22
EastSunsetWay               14
HighSchool                  14
MargaretsWay                10
ChiricoTrail_PooPooPoint     7
SquakMountain                6
MountTeneriffe               6
MountSi                      5
LittleSi                     1
Name: StopName, dtype: int64

In [27]:
dfk = pd.merge(
    left=trailheadHotspots,
    right=trailheadRef,
    how='left',
    left_on='StopName',
    right_on='StopName')

In [28]:
##calculate absolute distance from stop location
for i in dfk.itertuples():
    latInt = dfk['lat']
    latitudeInt = dfk['Latitude']
    lngInt = dfk['lng']
    longitudeInt = dfk['Longitude']
    dfk['dist_fromTH'] = abs(latitudeInt-latInt) + abs(longitudeInt-lngInt)
dfk['dist_fromTH'].astype('int32')

0      0
1      0
2      0
3      0
4      0
      ..
106    0
107    0
108    0
109    0
110    0
Name: dist_fromTH, Length: 111, dtype: int32

In [30]:
##sort to push to the hotspot closest to the trailhead to the top of the subset
dfk.sort_values(by=['locName','dist_fromTH'],inplace=True)
##remove any duplicate hotspots except for the hotspot closest to the trailhead
dfk_cooked = dfk.drop_duplicates(subset=['locName'],keep='first')

dfk_cooked['StopName'].value_counts()

DiscoveryParkSouth          14
DiscoveryParkNorth          13
EastSunsetWay                9
MargaretsWay                 8
MountTeneriffe               5
HighSchool                   3
ChiricoTrail_PooPooPoint     2
SquakMountain                1
MountSi                      1
Name: StopName, dtype: int64